# DATA

# Import

In [ ]:
import os
import cv2
import numpy as np
import random
import pandas as pd
from itertools import combinations
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision import transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau


# Balanced Dataset (Pair for Siamese Network) Generation

Pairs formed (for both +ve and -ve)
- Normal-Normal
- Normal-Distorted
- Distorted-Distorted


All possible +ve pairs formed then -ve pairs formed accordingly to make balanced dataset


In [ ]:


def load_images_from_folder(folder, image_size):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):  # Ensure only images
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, image_size)
                images.append(img)
    return images

def create_pairs_with_distortions_balanced(base_path, save_dir, image_size=(128, 128)):
    os.makedirs(save_dir, exist_ok=True)
    os.makedirs(os.path.join(save_dir, "x1"), exist_ok=True)
    os.makedirs(os.path.join(save_dir, "x2"), exist_ok=True)

    people = [p for p in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, p))]
    pair_records = []
    pair_id = 0
    positive_pairs = []

    print("Generating positive pairs...")

    # positive pairs
    person_to_images = {}
    for person in tqdm(people):
        person_path = os.path.join(base_path, person)
        normal_images = load_images_from_folder(person_path, image_size)
        distorted_path = os.path.join(person_path, "distorted")
        distorted_images = []
        if os.path.exists(distorted_path):
            distorted_images = load_images_from_folder(distorted_path, image_size)

        all_images = normal_images + distorted_images
        person_to_images[person] = all_images

        for img1, img2 in combinations(all_images, 2):
            positive_pairs.append((img1, img2))

    num_positive = len(positive_pairs)
    print(f"Total positive pairs: {num_positive}")

    #Save positive pairs
    for img1, img2 in positive_pairs:
        x1_path = os.path.join("x1", f"pair_{pair_id}.jpg")
        x2_path = os.path.join("x2", f"pair_{pair_id}.jpg")
        cv2.imwrite(os.path.join(save_dir, x1_path), img1)
        cv2.imwrite(os.path.join(save_dir, x2_path), img2)
        pair_records.append([x1_path, x2_path, 1])
        pair_id += 1

    # Generate negative pairs equal in number to positive pairs
    print("Generating balanced negative pairs...")
    negative_pairs = []
    attempts = 0
    max_attempts = num_positive * 10

    while len(negative_pairs) < num_positive and attempts < max_attempts:
        person1, person2 = random.sample(people, 2)
        imgs1 = person_to_images[person1]
        imgs2 = person_to_images[person2]

        if not imgs1 or not imgs2:
            attempts += 1
            continue

        img1 = random.choice(imgs1)
        img2 = random.choice(imgs2)

        negative_pairs.append((img1, img2))
        attempts += 1


    for img1, img2 in negative_pairs:
        x1_path = os.path.join("x1", f"pair_{pair_id}.jpg")
        x2_path = os.path.join("x2", f"pair_{pair_id}.jpg")
        cv2.imwrite(os.path.join(save_dir, x1_path), img1)
        cv2.imwrite(os.path.join(save_dir, x2_path), img2)
        pair_records.append([x1_path, x2_path, 0])
        pair_id += 1


    df = pd.DataFrame(pair_records, columns=["img1", "img2", "label"])
    df.to_csv(os.path.join(save_dir, "pairs_labels.csv"), index=False)
    print(f"Total pairs saved: {len(df)} (Positive: {df['label'].sum()}, Negative: {(df['label']==0).sum()})")


create_pairs_with_distortions_balanced(
    "/kaggle/input/comys-hackathon5-2025/Comys_Hackathon5/Task_B/train",
    "/kaggle/working/train_pairs"
)

create_pairs_with_distortions_balanced(
    "/kaggle/input/comys-hackathon5-2025/Comys_Hackathon5/Task_B/val",
    "/kaggle/working/val_pairs"
)


Generating positive pairs...


100%|██████████| 877/877 [00:22<00:00, 39.32it/s]


Total positive pairs: 34016
Generating balanced negative pairs...
Total pairs saved: 68032 (Positive: 34016, Negative: 34016)
Generating positive pairs...


100%|██████████| 250/250 [00:04<00:00, 50.32it/s]


Total positive pairs: 619
Generating balanced negative pairs...
Total pairs saved: 1238 (Positive: 619, Negative: 619)


Data Loader

In [ ]:


class FacePairsDataset(Dataset):
    def __init__(self, csv_path, base_dir, image_size=(128,128), augment=False):
        self.df = pd.read_csv(csv_path)
        self.base_dir = base_dir
        self.image_size = image_size
        self.augment = augment

        self.transform = A.Compose([
            A.Resize(*image_size),
            A.HorizontalFlip(p=0.5),
            A.Rotate(limit=15, p=0.5),
            A.Normalize(),
            ToTensorV2()
                ]) if augment else A.Compose([
                    A.Resize(*image_size),
                    A.Normalize(),
                    ToTensorV2()
                ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img1_path = os.path.join(self.base_dir, row['img1'])
        img2_path = os.path.join(self.base_dir, row['img2'])

        img1 = cv2.imread(img1_path)
        img2 = cv2.imread(img2_path)

        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
        img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

        img1 = self.transform(image=img1)['image']
        img2 = self.transform(image=img2)['image']

        label = torch.tensor(row['label'], dtype=torch.float32)

        return img1, img2, label




/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
train_dataset = FacePairsDataset(
    "/kaggle/working/train_pairs/pairs_labels.csv",
    "/kaggle/working/train_pairs",
    image_size=(128, 128),
    augment=True
)

val_dataset = FacePairsDataset(
    "/kaggle/working/val_pairs/pairs_labels.csv",
    "/kaggle/working/val_pairs",
    image_size=(128, 128),
    augment=False
)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)



# Model Architecture

# Model Explanation: MSFF_WinAttn_MobileNet_Embedding

This model is designed to produce **compact embedding vectors** from input images, suitable for applications like:

 Face or object embedding  
 Image retrieval  
 Metric learning

It combines the strength of a pretrained **MobileNetV2** backbone with additional feature fusion and attention mechanisms.

---

##  **Model Components**

###  1. MobileNetV2 Backbone
- The pretrained MobileNetV2 network is split into 4 sequential stages:
  - **Stage 1**: Early low-level features.
  - **Stage 2**: Intermediate features.
  - **Stage 3**: Deeper semantic features.
  - **Stage 4**: High-level features.
- Each stage produces feature maps of different resolution and semantics.

---

###  2. Feature Reduction
- Each stage output is reduced to **256 channels** using a `1×1` convolution.
- This ensures a consistent dimension across all stages before fusion.

---

###  3. Global Pooling & Dropout
- Each reduced feature map is **globally average pooled** to a vector of shape `(batch_size, 256)`.
- Dropout regularization is applied to prevent overfitting.

---

###  4. Multi-Stage Feature Fusion (MSFF)
- The four pooled vectors are **stacked along a new dimension**:


In [ ]:
class DropPath(nn.Module):
    def __init__(self, drop_prob=0.):
        super().__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        if self.drop_prob == 0. or not self.training:
            return x
        keep_prob = 1 - self.drop_prob
        shape = (x.shape[0],) + (1,) * (x.ndim - 1)
        random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
        random_tensor.floor_()
        return x.div(keep_prob) * random_tensor

class WindowAttention(nn.Module):
    def __init__(self, dim, heads=4):
        super().__init__()
        self.heads = heads
        self.scale = (dim // heads) ** -0.5
        self.to_qkv = nn.Linear(dim, dim * 3, bias=False)
        self.to_out = nn.Linear(dim, dim)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.to_qkv(x).chunk(3, dim=-1)
        q, k, v = map(
            lambda t: t.view(B, N, self.heads, C // self.heads).transpose(1, 2),
            qkv
        )
        dots = torch.matmul(q, k.transpose(-2, -1)) * self.scale
        attn = dots.softmax(dim=-1)
        out = torch.matmul(attn, v)
        out = out.transpose(1, 2).reshape(B, N, C)
        return self.to_out(out)

class MSFF_WinAttn_MobileNet_Embedding(nn.Module):
    def __init__(self, embedding_dim=128, drop_path_prob=0.1):
        super().__init__()
        mobilenet = models.mobilenet_v2(pretrained=True).features

        self.stage1 = mobilenet[:4]
        self.stage2 = mobilenet[4:7]
        self.stage3 = mobilenet[7:14]
        self.stage4 = mobilenet[14:]

        self.reduce1 = nn.Conv2d(24, 256, 1)
        self.reduce2 = nn.Conv2d(32, 256, 1)
        self.reduce3 = nn.Conv2d(96, 256, 1)
        self.reduce4 = nn.Conv2d(1280, 256, 1)

        self.attn1 = WindowAttention(256, heads=4)
        self.bn1 = nn.BatchNorm1d(256)
        self.drop_path1 = DropPath(drop_path_prob)

        self.attn2 = WindowAttention(256, heads=4)
        self.bn2 = nn.BatchNorm1d(256)
        self.drop_path2 = DropPath(drop_path_prob)

        self.embed_fc = nn.Linear(256 * 4, embedding_dim)

        self.dropout = nn.Dropout(0.3)
        self.bn_final = nn.BatchNorm1d(256 * 4)


    def forward(self, x):
        x1 = self.stage1(x)
        x2 = self.stage2(x1)
        x3 = self.stage3(x2)
        x4 = self.stage4(x3)

        x1 = self.dropout(F.adaptive_avg_pool2d(self.reduce1(x1), 1).flatten(1))
        x2 = self.dropout(F.adaptive_avg_pool2d(self.reduce2(x2), 1).flatten(1))
        x3 = self.dropout(F.adaptive_avg_pool2d(self.reduce3(x3), 1).flatten(1))
        x4 = self.dropout(F.adaptive_avg_pool2d(self.reduce4(x4), 1).flatten(1))


        feats = torch.stack([x1, x2, x3, x4], dim=1)
        feats = self.attn1(feats)
        feats = self.drop_path1(feats)
        B, N, C = feats.shape
        feats = feats.view(B * N, C)
        feats = self.bn1(feats)
        feats = F.relu(feats)
        feats = feats.view(B, N, C)

        feats = self.attn2(feats)
        feats = self.drop_path2(feats)
        feats = feats.view(B * N, C)
        feats = self.bn2(feats)
        feats = F.relu(feats)
        feats = feats.view(B, N, C)

        out = feats.flatten(1)
        # embed = self.embed_fc(out)
        out = self.dropout(out)
        out = F.relu(out)
        out = self.bn_final(out)
        embed = self.embed_fc(out)

        embed = F.normalize(embed, p=2, dim=1)
        return embed

In [ ]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super().__init__()
        self.margin = margin

    def forward(self, emb1, emb2, label):
        dist = F.pairwise_distance(emb1, emb2)
        loss_same = label * dist.pow(2)
        loss_diff = (1 - label) * F.relu(self.margin - dist).pow(2)
        return 0.5 * (loss_same + loss_diff).mean()


In [ ]:
model = MSFF_WinAttn_MobileNet_Embedding(embedding_dim=128).cuda()
criterion = ContrastiveLoss(margin=1.0)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 10

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 117MB/s] 


In [ ]:
model = MSFF_WinAttn_MobileNet_Embedding(embedding_dim=128)



model = model.cuda()
criterion = ContrastiveLoss(margin=1.0)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)


num_epochs = 30


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


# Training

In [ ]:
best_val_loss = None
threshold = 0.5  # Distance threshold

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")

    # Training
    model.train()
    train_loss = 0
    all_train_preds = []
    all_train_labels = []
    train_batches = tqdm(train_loader, desc="Training", leave=False)

    for img1, img2, labels in train_batches:
        img1 = img1.cuda()
        img2 = img2.cuda()
        labels = labels.cuda()

        emb1 = model(img1)
        emb2 = model(img2)

        loss = criterion(emb1, emb2, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_batches.set_postfix(loss=loss.item())

        # Collect predictions for metrics
        dists = F.pairwise_distance(emb1, emb2)
        preds = (dists < threshold).long()
        all_train_preds.extend(preds.cpu().numpy())
        all_train_labels.extend(labels.cpu().numpy())

    avg_train_loss = train_loss / len(train_loader)
    all_train_preds = np.array(all_train_preds)
    all_train_labels = np.array(all_train_labels)
    train_accuracy = (all_train_preds == all_train_labels).mean()
    train_f1 = f1_score(all_train_labels, all_train_preds, average="macro")

    #Validation
    model.eval()
    val_loss = 0
    all_val_preds = []
    all_val_labels = []
    val_batches = tqdm(val_loader, desc="Validation", leave=False)

    with torch.no_grad():
        for img1, img2, labels in val_batches:
            img1 = img1.cuda()
            img2 = img2.cuda()
            labels = labels.cuda()

            emb1 = model(img1)
            emb2 = model(img2)

            loss = criterion(emb1, emb2, labels)
            val_loss += loss.item()

            dists = F.pairwise_distance(emb1, emb2)
            preds = (dists < threshold).long()

            all_val_preds.extend(preds.cpu().numpy())
            all_val_labels.extend(labels.cpu().numpy())
            val_batches.set_postfix(loss=loss.item())

    avg_val_loss = val_loss / len(val_loader)
    all_val_preds = np.array(all_val_preds)
    all_val_labels = np.array(all_val_labels)
    val_accuracy = (all_val_preds == all_val_labels).mean()
    val_f1 = f1_score(all_val_labels, all_val_preds, average="macro")

    # Save best model
    if best_val_loss is None or avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), "best_model.pth")

    scheduler.step(avg_val_loss)


    print(
        f"Epoch {epoch+1}/{num_epochs} "
        f"| Train Loss: {avg_train_loss:.4f} "
        f"| Train Acc: {train_accuracy:.4f} "
        f"| Train F1: {train_f1:.4f} "
        f"| Val Loss: {avg_val_loss:.4f} "
        f"| Val Acc: {val_accuracy:.4f} "
        f"| Val F1: {val_f1:.4f}"
    )



Epoch 1/30


Epoch 1/30 | Train Loss: 0.0573 | Train Acc: 0.9010 | Train F1: 0.9010 | Val Loss: 0.1178 | Val Acc: 0.6866 | Val F1: 0.6717

Epoch 2/30


Epoch 2/30 | Train Loss: 0.0263 | Train Acc: 0.9481 | Train F1: 0.9481 | Val Loss: 0.0865 | Val Acc: 0.7544 | Val F1: 0.7508

Epoch 3/30


Epoch 3/30 | Train Loss: 0.0193 | Train Acc: 0.9567 | Train F1: 0.9567 | Val Loss: 0.1077 | Val Acc: 0.6535 | Val F1: 0.6282

Epoch 4/30


Epoch 4/30 | Train Loss: 0.0176 | Train Acc: 0.9592 | Train F1: 0.9592 | Val Loss: 0.0870 | Val Acc: 0.7399 | Val F1: 0.7320

Epoch 5/30


...

Epoch 26/30 | Train Loss: 0.0118 | Train Acc: 0.9717 | Train F1: 0.9717 | Val Loss: 0.0913 | Val Acc: 0.7270 | Val F1: 0.7152

Epoch 27/30


Epoch 27/30 | Train Loss: 0.0116 | Train Acc: 0.9719 | Train F1: 0.9719 | Val Loss: 0.0829 | Val Acc: 0.7472 | Val F1: 0.7387

Epoch 28/30


Epoch 28/30 | Train Loss: 0.0119 | Train Acc: 0.9710 | Train F1: 0.9710 | Val Loss: 0.0860 | Val Acc: 0.7407 | Val F1: 0.7322

Epoch 29/30


Epoch 29/30 | Train Loss: 0.0116 | Train Acc: 0.9723 | Train F1: 0.9723 | Val Loss: 0.0843 | Val Acc: 0.7431 | Val F1: 0.7355

Epoch 30/30


Epoch 30/30 | Train Loss: 0.0115 | Train Acc: 0.9717 | Train F1: 0.9717 | Val Loss: 0.0979 | Val Acc: 0.7108 | Val F1: 0.6966


# Testing Phase

In [ ]:
def test_model(model, model_path, test_folder, device='cuda' if torch.cuda.is_available() else 'cpu', batch_size=32):
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()

    create_pairs_with_distortions_balanced(test_folder,"./test_pairs")
    test_dataset = FacePairsDataset("./test_pairs/pairs_labels.csv","./test_pairs",image_size=(128, 128),augment=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    all_preds = []
    all_labels = []
    val_loss = 0.0
    test_batches = tqdm(test_loader, desc="test", leave=False)

    with torch.no_grad():
        for img1, img2, labels in test_batches:
            img1 = img1.cuda()
            img2 = img2.cuda()
            labels = labels.cuda()

            emb1 = model(img1)
            emb2 = model(img2)

            loss = criterion(emb1, emb2, labels)
            val_loss += loss.item()

            dists = F.pairwise_distance(emb1, emb2)
            preds = (dists < threshold).long()

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    rec = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

    print(f"\n Test Accuracy: {acc:.4f}")
    print(f" Precision:     {prec:.4f}")
    print(f" Recall:        {rec:.4f}")
    print(f" F1 Score:      {f1:.4f}")

    return acc, prec, rec, f1




In [ ]:
test_folder = "/kaggle/input/comys-hackathon5-2025/Comys_Hackathon5/Task_B/val"
model_path = "/kaggle/working/best_model.pth"

acc, prec, rec, f1 = test_model(model, model_path, test_folder)

Generating positive pairs...


100%|██████████| 250/250 [00:02<00:00, 90.90it/s]


Total positive pairs: 619
Generating balanced negative pairs...
Total pairs saved: 1238 (Positive: 619, Negative: 619)



 Test Accuracy: 0.7593
 Precision:     0.7850
 Recall:        0.7593
 F1 Score:      0.7537


In [ ]:
test_folder = "/content/extracted_folder/Comys_Hackathon5/Task_B/train"
model_path = "/content/best_model.pth"

acc, prec, rec, f1 = test_model(model, model_path, test_folder)

Generating positive pairs...


100%|██████████| 877/877 [00:06<00:00, 131.99it/s]


Total positive pairs: 34016
Generating balanced negative pairs...
Total pairs saved: 68032 (Positive: 34016, Negative: 34016)



 Test Accuracy: 0.9745
 Precision:     0.9755
 Recall:        0.9745
 F1 Score:      0.9745
